# Итоговый проект (урок № 9), Кривоногов Н.В.

## Step 3: FLASK

In [1]:
# !pip install flask==2.1.0

In [2]:
# from flask_ngrok import run_with_ngrok # для google colab

from flask import Flask, request, jsonify
import pandas as pd
import dill

### Создаю сервис для обработки запросов к модели

In [3]:
# загружаю обученную модель: 

with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [4]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

#### Запускаю сервис и не глушу его, пока работаю 

In [8]:
# обработчики и запуск Flask: 

app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run # для google colab


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    convex_area, eccentricity, perimeter = "", "", ""
    request_json = request.get_json()

    if request_json["convex_area"]:
        convex_area = request_json['convex_area']

    if request_json["eccentricity"]:
        eccentricity = request_json['eccentricity']
        
    if request_json["perimeter"]:
        perimeter = request_json['perimeter']
    
    print(convex_area, eccentricity, perimeter)  
    preds = model.predict_proba(pd.DataFrame({"convex_area": [convex_area],
                                              "eccentricity": [eccentricity],
                                              "perimeter": [perimeter]}))
    data["predictions"] = preds[:, 1][0]
    # indicate that the request was a success
    data["success"] = True
    print('OK')

    # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


### Тестовый клиент

In [9]:
# пример данных:

convex_area, eccentricity, perimeter = (16343, 0.90705806, 518.8930054)

body = {
        'convex_area': convex_area,
        'eccentricity': eccentricity,
        'perimeter': perimeter
        }

In [10]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

16343 0.90705806 518.8930054
OK


{'predictions': 0.9993726893179599, 'success': True}